In [1]:
from open3d import *
import numpy as np
from scipy.io import loadmat, savemat

import os
from os import listdir
from os.path import isfile, join

In [2]:
def clean_labels(labels):
    
    new_labels = []
    
    for i in range (0, len(labels)):
                
        for j in range (0, len(labels[i])):
                    
            if(labels[i][j] > 0):
                        
                new_labels.append(labels[i][j])
                
    return new_labels

In [3]:
# removes background of the depth image using the point_labels (0 = background)

def remove_background(depth, point_labels):
    np_depth = np.asarray(depth)
    new_depth = np.zeros((480, 640))
    
    for i in range (0, len(point_labels)):
        
        for j in range (0, len(point_labels[i])):
            
            if(point_labels[i][j] != 0):
                
                new_depth[i][j] = np_depth[i][j]
            
            else:
                
                new_depth[i][j] = 0
        
    return new_depth

In [4]:
# load the directory names from a text file

def load_directory_names(filename):
    
    directory_list = []
    
    file = open(filename, 'r') 
    lines = file.readlines() 

    for line in lines: 
        line_string = line.strip()
        split_string = line_string.split(" ", 2)
        
        directory_list.append(split_string[1])
        
    return directory_list

In [5]:
# get all filenames in a given folder 

def get_all_filenames_from_directory(directory_path):
    onlyfiles = [f for f in listdir(directory_path) if isfile(join(directory_path, f))]
    
    return onlyfiles

In [6]:
def create_rgb_image_list_from_directory(directory_path):
    
    filenames = get_all_filenames_from_directory(directory_path)
    filenames.sort()
    
    rgb_list = [];
    
    suffix = "_rgb.jpg"
    
    for file in filenames:
        
        if(file.endswith(suffix)):
            rgb_list.append(file)
            
    
    rgb_list.sort()
    
    return rgb_list

In [7]:
def create_depth_image_list_from_directory(directory_path):
    
    filenames = get_all_filenames_from_directory(directory_path)
    filenames.sort()
    
    rgb_list = [];
    
    suffix = "_depth.png"
    
    for file in filenames:
        
        if(file.endswith(suffix)):
            rgb_list.append(file)
            
    
    rgb_list.sort()
    
    return rgb_list

In [8]:
def create_label_list_from_directory(directory_path):
    
    filenames = get_all_filenames_from_directory(directory_path)
    filenames.sort()
    
    rgb_list = [];
    
    suffix = "_label.mat"
    
    for file in filenames:
        
        if(file.endswith(suffix)):
            rgb_list.append(file)
            
    
    rgb_list.sort()
    
    return rgb_list

In [9]:
def tranform_rgbd_images_to_point_clouds(label_list, depth_list, rgb_list, path_to_data, database_path):
    
    if((len(depth_list) == len(rgb_list)) and (len(depth_list) == len(label_list))):
        
        split_string = label_list[0].split('_')
        
        new_folder_path = database_path + '/' + split_string[0] + '_' + split_string[1]
        
        # create new folder to save the point cloud wich is generated
        if not os.path.exists(new_folder_path):
            os.makedirs(new_folder_path)
            
        point_error = False
        
        # loop over all list entries
        for i in range (0, len(depth_list)):
            
            # load labels for the specific image
            labels = loadmat(path_to_data + '/' + label_list[i])
            
            # load color and depth image
            color = open3d.io.read_image(path_to_data + '/' + rgb_list[i])
            depth = open3d.io.read_image(path_to_data + '/' + depth_list[i])
            
            # remove background from depth image
            new_depth = remove_background(depth, labels['gt_label'])
            
            # remove background labels
            new_labels = clean_labels(labels['gt_label'])
            save_dict = {"data" : new_labels}
            
            # transform new depth image from numpy array to open3d image
            depth = open3d.geometry.Image(np.array(new_depth).astype('uint16'))
            
            # intrinsic camera
            intrinsic = open3d.camera.PinholeCameraIntrinsic(open3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)
            
            rgbd = create_rgbd_image_from_color_and_depth(color, depth, convert_rgb_to_intensity = False)
            pcd = create_point_cloud_from_rgbd_image(rgbd, intrinsic)

            # flip the orientation, so it looks upright, not upside-down
            pcd.transform([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])
            
            # draw_geometries([pcd]) #vizualize resulting point cloud
            
            # save point cloud in new generated folder
            open3d.io.write_point_cloud(new_folder_path + '/' + str(i) + '.pcd', pcd)
            
            # save new labels without background
            savemat(new_folder_path + '/' + str(i) + '.mat', save_dict)
            
            # open saved files again and check if number of labels and number of points in pointcloud are the same
            saved_labels = loadmat(new_folder_path + '/' + str(i) + '.mat')['data']
            saved_pcd = open3d.io.read_point_cloud(new_folder_path + '/' + str(i) + '.pcd')
            
            if(len(saved_labels[0]) != len(np.asarray(saved_pcd.points))):
                print("Error #labels and #points in pointcloud do not match")
                os.remove(new_folder_path + '/' + str(i) + '.mat')
                print("Removed .mat file: " + new_folder_path + '/' + str(i) + '.mat')
                os.remove(new_folder_path + '/' + str(i) + '.pcd')
                print("Removed .pcd file: " + new_folder_path + '/' + str(i) + '.pcd')
                point_error = True
                
        if(point_error == False):
            print("No error ocurred in folder: " + split_string[0] + '_' + split_string[1])


    else:
        print("Depth " + str(len(depth_list)))
        print("RGB " + str(len(rgb_list)))
        print("Label " + str(len(label_list)))
        print("Error transforming: list sizes are not the same")

In [10]:
# main


# get current working directory
current_working_directory = os.getcwd()

database_path = current_working_directory + "/dataset"

# create new database folder
if not os.path.exists(database_path):
    os.makedirs(database_path)


# load the file where all the directories are mentioned
directory_list = load_directory_names('./tool_categories.txt')


# iterate over all directories
for folder_name in directory_list:
    
    tmp_dir = current_working_directory + "/" + folder_name
    
    # get list of label, rgb images and depth images
    label_list = create_label_list_from_directory(tmp_dir)
    depth_list = create_depth_image_list_from_directory(tmp_dir)
    rgb_list = create_rgb_image_list_from_directory(tmp_dir)
    
    # transform rgb-d image to point cloud and save it in a new folder in the database folder
    tranform_rgbd_images_to_point_clouds(label_list, depth_list, rgb_list, current_working_directory + '/' + folder_name, database_path)
    
    

Error #labels and #points in pointcloud do not match
Removed .mat file: /Users/manuel/Desktop/part_affordance/dataset/hammer_01/0.mat
Removed .pcd file: /Users/manuel/Desktop/part_affordance/dataset/hammer_01/0.pcd
Error #labels and #points in pointcloud do not match
Removed .mat file: /Users/manuel/Desktop/part_affordance/dataset/hammer_01/2.mat
Removed .pcd file: /Users/manuel/Desktop/part_affordance/dataset/hammer_01/2.pcd
Error #labels and #points in pointcloud do not match
Removed .mat file: /Users/manuel/Desktop/part_affordance/dataset/hammer_01/3.mat
Removed .pcd file: /Users/manuel/Desktop/part_affordance/dataset/hammer_01/3.pcd
